# 構造化データ： NumPyの構造化array

ここで、何人かの人のいくつかのカテゴリーに分かれたデータを持っているとする。例えば、名前や年齢、体重である。それらをPythonのプログラムで保持したいと考える。それらを3つの別々のarrayで保持することは可能であろう。

In [1]:
import numpy as np

In [2]:
name = ['Alice', 'Bob', 'Cathy', 'Doug']
age = [25, 45, 37, 19]
weight = [55.0, 85.5, 68.0, 61.5]

しかし、これをすこし不格好である。その3つのarrayが関係しているかどうかはここでは分からない。これらのデータを一つの構造化データとして扱うことできれば、より自然である。

NumPyでは構造化したarrayを扱うことが出来る。

ここで、以前次のような式を利用してシンプルらarrayを作成したことを思い出す。

In [3]:
x = np.zeros(4, dtype=int)

上記と似た形式で、合成したデータタイプの記述を利用して構造化arrayを作成することが出来る。

In [4]:
data = np.zeros(4, dtype={'names':('name', 'age', 'weight'),
                          'formats':('U10', 'i4', 'f8')})
print(data.dtype)

[('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]


ここで、以下はそれぞれ次のような意味を持つ。

- `U10`: Unicode string of maximum length 10
- `i4`: 4-byte (i.e., 32 bit) integer
- `f8`: 8-byte (i.e., 64 bit) float

ここまでで、空のコンテナのarrayが作成されている。リストの値によってarrayを埋めることが出来る。

In [5]:
data['name'] = name
data['age'] = age
data['weight'] = weight
print(data)

[('Alice', 25, 55. ) ('Bob', 45, 85.5) ('Cathy', 37, 68. )
 ('Doug', 19, 61.5)]


それぞれの値をインデックスや名前でアクセスすることが出来るようになった。

In [6]:
data['name']

array(['Alice', 'Bob', 'Cathy', 'Doug'], dtype='<U10')

In [7]:
data[0]

('Alice', 25, 55.)

In [8]:
data[-1]['name']

'Doug'

Boolean maskを利用して、年齢でのfilteringなどが可能となる。

In [9]:
data[data['age'] < 30]['name']

array(['Alice', 'Doug'], dtype='<U10')

ただし、Pandasにも`Dataframe`オブジェクトという便利なものがあるので、こちらも確認しておいた方がよい。

## 構造化arrayの作成

In [10]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':('U10', 'i4', 'f8')})

dtype([('name', '<U10'), ('age', '<i4'), ('weight', '<f8')])

Pythonの型や、NumPyの`dtype`を代わりに指定することも出来る。

In [11]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':((np.str_, 10), int, np.float32)})

dtype([('name', '<U10'), ('age', '<i8'), ('weight', '<f4')])

タプルのリストとして指定することも可能。

In [12]:
np.dtype([('name', 'S10'), ('age', 'i4'), ('weight', 'f8')])

dtype([('name', 'S10'), ('age', '<i4'), ('weight', '<f8')])

型の名前が何だっていい場合には、型のみの指定だけでも作成可能。

In [13]:
np.dtype('S10,i4,f8')

dtype([('f0', 'S10'), ('f1', '<i4'), ('f2', '<f8')])

型のフォーマットはendian、data type、byte sizeを示している。

`<`はリトルエンディアンを`>`はビッグエンディアンを表す。

data typeは以下の表の通りである。

byte sizeはオブジェクトのbyteでのサイズを示す。

|Character|Description|Example|
|:--------|:----------|:------|
|'b'|Byte|np.dtype('b')|
|'i'|Signed integer|np.dtype('i4') == np.int32|
|'u'|Unsignedinteger|np.dtype('u1') == np.uint8|
|'f'|Floating point|np.dtype('f8') == np.int64|
|'c'|Complex floating point|np.dtype('c16') == np.complex128|
|'S', 'a'|String|np.dtype('S5')|
|'U'|Unicode string|np.dtype('U') == np.str_|
|'V'|Raw data (void)|np.dtype('V') == np.void|

## より応用的な型
次のような複雑なものを作成することも出来る。

例えば、古いCのライブラリとのインターフェースが必要な時などに便利である。

In [14]:
tp = np.dtype([('id', 'i8'), ('mat', 'f8', (3, 3))])
X = np.zeros(1, dtype=tp)
print(X[0])
print(X['mat'][0])

(0, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


## RecordArrays: Structured Arrays with a Twist
NumPyは`nprecarray`クラスも提供している。これは構造化arrayとほぼ同じだが、フィールドに対して辞書のキーとしてアクセスするのでなく、属性としてアクセスすることが可能となる。

In [15]:
data['age']

array([25, 45, 37, 19], dtype=int32)

代わりにレコードアレイとしてデータを見れば、以下の通りアクセスできる。

In [16]:
data_rec = data.view(np.recarray)
data_rec.age

array([25, 45, 37, 19], dtype=int32)

ただし、フィールドにアクセスするのに余分なオーバーヘッドが加わるため、アクセス速度は遅くなる。

In [17]:
%timeit data['age']
%timeit data_rec['age']
%timeit data_rec.age

131 ns ± 1.59 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
2.82 µs ± 30 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.6 µs ± 26.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
